<a href="https://colab.research.google.com/github/Mogarthron/Magazynier/blob/master/DOSTAWY_PIANEK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

from datetime import datetime as dt, timedelta
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

In [ ]:
engine = create_engine("sqlite:///drive/MyDrive/GENERATORY.db", echo=False)
with engine.begin() as conn:
      tab = pd.read_sql(text("SELECT * from baza_PIANKI"), conn)
      zp_tab = pd.read_sql(text("SELECT * from ZAM_PIANKI"), conn)# WHERE STATUS_KOMPLETACJA <> '1'
zp_tab["KOMPLETACJA"] = zp_tab["MODEL"] + " " + zp_tab["NR_KOMPLETACJI"]
zp_tab["nr_SAMOCHODU"].fillna("", inplace=True)
zp_tab = zp_tab[zp_tab.STATUS_KOMPLETACJA !="1"]

tab["vol"] = tab.DLUG*tab.SZER*tab.WYS/1000_000_000
tab["VOL"] = tab.vol*tab.ilosc


In [ ]:
zp_tab

,LP,TYDZIEN,KOD,MODEL,NR_KOMPLETACJI,OPIS,ILE_ZAMOWIONE,ZNACZNIK_DOSTAWCY,GALANTERIA,SIEDZISKA_HR,...,DATA_DOSTARCZENIA_1,DATA_DOSTARCZENIA_2,STATUS_LENIWA,STATUS_LENIWA_SKOSOWANIE,STATUS_OWATY,nr_PW,STATUS_KOMPLETACJA,nr_PZ,nr_SAMOCHODU,KOMPLETACJA
2,3,2401,16.131.53083.60,CALYPSO,1_24,CALYPSO [LAD,10,C,C,C,...,2024-02-12,None,None,None,None,None,C,24/0027,CIECH 8_24,CALYPSO 1_24
3,4,2401,16.131.65004.60,CALYPSO,1_24,CALYPSO [L2H,10,C,C,C,...,2024-02-12,None,None,None,None,None,C,24/0027,CIECH 8_24,CALYPSO 1_24
4,5,2401,16.131.00001.60,CALYPSO,1_24,CALYPSO ][,60,C,C,C,...,2024-02-12,None,None,None,None,None,C,24/0027,CIECH 8_24,CALYPSO 1_24
16,17,2402,16.105.15200.60,HORIZON,1_24,"HORIZON 1,5z",90,CV,V,V,...,2024-02-12,2024-02-02,None,None,None,None,CV,24/0024,"CIECH 8_24,VITA 5_24",HORIZON 1_24
17,18,2402,16.105.68000.60,HORIZON,1_24,HORIZON Onz,10,CV,V,V,...,2024-02-12,2024-02-02,None,None,None,None,CV,24/0024,"CIECH 8_24,VITA 5_24",HORIZON 1_24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,206,2408,16.125.71600.60,STONE,2_24,STONE P106x106,10,V,V,V,...,None,None,None,None,None,None,None,None,,STONE 2_24
204,207,2408,16.125.71900.60,STONE,2_24,STONE P106x82,20,V,V,V,...,None,None,None,None,None,None,None,None,,STONE 2_24
205,208,2408,16.125.67300.60,STONE,2_24,STONE Z70,190,V,V,V,...,None,None,None,None,None,None,None,None,,STONE 2_24
206,209,2408,16.125.00004.60,STONE,2_24,STONE ][,250,V,V,V,...,None,None,None,None,None,None,None,None,,STONE 2_24


In [ ]:
komplety_pianek = pd.read_excel("drive/MyDrive/ZAM_PIANKI.xlsx", sheet_name="Arkusz3")
komplety_pianek['CZY_BRYLA'] = komplety_pianek['CZY_BRYLA'].fillna(1)
komplety_pianek['BRYLA_GEN'] = komplety_pianek['BRYLA_GEN'].fillna("").astype(str).apply(lambda x: x.replace(".", ","))

aktywne_dos = pd.read_excel("drive/MyDrive/ZAM_PIANKI.xlsx", sheet_name="AKTYWNE_DOS")

In [ ]:
def obietosci_samochodow(dostawca, tabela):

  dostawy = tabela[tabela.nr_SAMOCHODU.str.contains(dostawca)]

  dostawy["SAMOCHOD"] = dostawy.nr_SAMOCHODU.apply(lambda x: [d for d in x.split(",") if dostawca in d][0])

  dostawy = dostawy.merge(komplety_pianek[["KOD", "BRYLA_GEN"]], how="left", on="KOD")

  def obj_typ(m,bg,g,s,l,i,t):
    war_len = (tab.TYP == "G-401")
    war_shr = (tab.TYP.str.contains("HR|EE"))

    if t == "g":
      return tab[(tab.MODEL == m) & (tab.BRYLA == bg) & ~war_len & ~war_shr].VOL.sum()*i if g == dostawca[0] else 0
    if t == "s":
      return tab[(tab.MODEL == m) & (tab.BRYLA == bg) & (war_shr)].VOL.sum()*i if s == dostawca[0] else 0
    if t == "l":
      return tab[(tab.MODEL == m) & (tab.BRYLA == bg) & (war_len)].VOL.sum()*i if l == dostawca[0] else 0

    # return f"{gal} + {shr} + {len}"
    # return gal + shr + len

  dostawy["GAL_OBJ"] = dostawy.apply(lambda x: obj_typ(x.MODEL, x.BRYLA_GEN, x.GALANTERIA, x.SIEDZISKA_HR, x.LENIWA, x.ILE_ZAMOWIONE, "g"), axis=1)
  dostawy["SHR_OBJ"] = dostawy.apply(lambda x: obj_typ(x.MODEL, x.BRYLA_GEN, x.GALANTERIA, x.SIEDZISKA_HR, x.LENIWA, x.ILE_ZAMOWIONE, "s"), axis=1)
  dostawy["LEN_OBJ"] = dostawy.apply(lambda x: obj_typ(x.MODEL, x.BRYLA_GEN, x.GALANTERIA, x.SIEDZISKA_HR, x.LENIWA, x.ILE_ZAMOWIONE, "l"), axis=1)
  dostawy["OBJ"] = dostawy.GAL_OBJ + dostawy.SHR_OBJ + dostawy.LEN_OBJ

  return dostawy[["KOMPLETACJA", "OPIS", 'ILE_ZAMOWIONE', 'SAMOCHOD', "GAL_OBJ", "SHR_OBJ", "LEN_OBJ", "OBJ"]]
  # return dostawy.groupby("SAMOCHOD").sum().OBJ, dostawy.groupby(["SAMOCHOD", "KOMPLETACJA"]).sum().OBJ

In [ ]:
aktywne_dos["ZAM"] = aktywne_dos["Nr zam."].str.replace("DOS/", "")

aktywne_dos["data_dos"] = aktywne_dos['Data dostawy'] + timedelta(1)

def nazwa_dostawcy(x):
  if "PIANPOL" in x:
    return "PIANPOL"
  if "VITA" in x:
    return "VITA"
  if "CIECH" in x:
    return "CIECH"

aktywne_dos["Dostawca"] = aktywne_dos["Dostawca opis"].apply(nazwa_dostawcy)

In [ ]:
obietosci_samochodow("CIECH", zp_tab).groupby(["SAMOCHOD", "KOMPLETACJA"]).sum()[["OBJ","GAL_OBJ","SHR_OBJ","LEN_OBJ"]].reset_index()

,SAMOCHOD,KOMPLETACJA,OBJ,GAL_OBJ,SHR_OBJ,LEN_OBJ
0,CIECH 2_23,REVERSO 2_23,2.161790,1.171790,0.000000,0.99000
1,CIECH 3_23,AMALFI 2_23,9.750460,0.000000,0.000000,9.75046
2,CIECH 3_24,DUO 1_23,0.978900,0.000000,0.000000,0.97890
3,CIECH 3_24,REVERSO 3_23,0.467820,0.467820,0.000000,0.00000
4,CIECH 6_24,ELIXIR 2_23,14.782460,8.763260,0.000000,6.01920
5,CIECH 6_24,WILLOW 2_23,17.525365,10.789315,0.000000,6.73605
6,CIECH 8_24,AMALFI 2_23,34.856699,29.767649,0.000000,5.08905
7,CIECH 8_24,CALYPSO 1_24,10.527955,4.839955,3.910500,1.77750
8,CIECH 8_24,HORIZON 1_24,10.160750,0.000000,0.000000,10.16075
9,CIECH 8_24\t,CALYPSO 2_23,5.187520,1.853400,2.309120,1.02500


In [ ]:
# aktywne_dos

In [ ]:
# fig = px.timeline(aktywne_dos, x_start="Data dostawy", x_end="data_dos", y="Dostawca", title="Daty realizacji zamówień", text="ZAM", hover_data=["Uwagi", "polaczona_z_dos"],  color="Dostawca")#, hover_data=["Opis"])
# fig.add_vline(x=dt.now(), line_width=.5, line_color="blue")
# fig.update_yaxes(autorange="reversed")
# fig.show()

df = pd.concat([
    obietosci_samochodow("CIECH", zp_tab).groupby(["SAMOCHOD", "KOMPLETACJA"]).sum()[["OBJ","GAL_OBJ","SHR_OBJ","LEN_OBJ"]].reset_index(),
    obietosci_samochodow("VITA", zp_tab).groupby(["SAMOCHOD", "KOMPLETACJA"]).sum()[["OBJ","GAL_OBJ","SHR_OBJ","LEN_OBJ"]].reset_index(),
    obietosci_samochodow("PIANPOL", zp_tab).groupby(["SAMOCHOD", "KOMPLETACJA"]).sum()[["OBJ","GAL_OBJ","SHR_OBJ","LEN_OBJ"]].reset_index()
]).sort_values(by="SAMOCHOD").reset_index(drop=True)

fig = px.bar(df, x="OBJ", y="SAMOCHOD", color='KOMPLETACJA', orientation='h',
             text="KOMPLETACJA",
             hover_data=["KOMPLETACJA", "GAL_OBJ","SHR_OBJ","LEN_OBJ"],
            #  height=400,
             title='Zapełnienie samochodów 2024-02-22')
fig.update_yaxes(autorange="reversed")
fig.update_layout(showlegend=False)
fig.add_vline(x=30, line_dash="dash", annotation_text="30m3")
fig.add_vline(x=60, line_dash="dash", annotation_text="60m3")
fig.add_vline(x=90, line_dash="dash", annotation_text="90m3")
fig.add_vline(x=100, line_color="red")
# fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

In [ ]:
df = pd.concat([
    obietosci_samochodow("CIECH", zp_tab).groupby(["SAMOCHOD", "KOMPLETACJA"]).sum()[["OBJ","GAL_OBJ","SHR_OBJ","LEN_OBJ"]].reset_index(),
    obietosci_samochodow("VITA", zp_tab).groupby(["SAMOCHOD", "KOMPLETACJA"]).sum()[["OBJ","GAL_OBJ","SHR_OBJ","LEN_OBJ"]].reset_index(),
    obietosci_samochodow("PIANPOL", zp_tab).groupby(["SAMOCHOD", "KOMPLETACJA"]).sum()[["OBJ","GAL_OBJ","SHR_OBJ","LEN_OBJ"]].reset_index()
])

In [ ]:
df.sort_values(by="SAMOCHOD").reset_index(drop=True)

,SAMOCHOD,KOMPLETACJA,OBJ,GAL_OBJ,SHR_OBJ,LEN_OBJ
0,CIECH 2_23,REVERSO 2_23,2.161790,1.171790,0.000000,0.990000
1,CIECH 3_23,AMALFI 2_23,9.750460,0.000000,0.000000,9.750460
2,CIECH 3_24,DUO 1_23,0.978900,0.000000,0.000000,0.978900
3,CIECH 3_24,REVERSO 3_23,0.467820,0.467820,0.000000,0.000000
4,CIECH 6_24,ELIXIR 2_23,14.782460,8.763260,0.000000,6.019200
5,CIECH 6_24,WILLOW 2_23,17.525365,10.789315,0.000000,6.736050
6,CIECH 8_24,AMALFI 2_23,34.856699,29.767649,0.000000,5.089050
7,CIECH 8_24,CALYPSO 1_24,10.527955,4.839955,3.910500,1.777500
8,CIECH 8_24,HORIZON 1_24,10.160750,0.000000,0.000000,10.160750
9,CIECH 8_24\t,CALYPSO 2_23,5.187520,1.853400,2.309120,1.025000
